In [1]:
import h5py
import numpy
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

C:\Users\Rakshith\Miniconda3\envs\machinelearning\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# check for GPU 
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11464384950194181277
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6707519816
locality {
  bus_id: 1
}
incarnation: 12719929708063655328
physical_device_desc: "device: 0, name: GeForce GTX 1070, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [3]:
# load ascii text and covert to lowercase
filename = "butterefly.txt"
raw_text = open(filename,encoding='utf-8').read()
raw_text = raw_text.lower()
raw_text

"nigga y.u mad i add the recipe to my raps you had your\nchance i use the trance to take shit to advance i'm\ntaking my stance i'm back on my gangsta flow\ni don't fuck with the spam damn i be showing y'all a good t\ntime this gonn be a summertime jam yall take exams on\nmy swag cause i'm gonna drag y'all to the bottom\ny'all smell like rotten milk i killed y'all motto i killed the lotto\nthrow sum mo hundreds throw that pussy back bitch i\nswitch up my flow cause i'm one of a kind i be on my\nmastermind grind i got rid of the whack niggas that's y.u\nmad that's y.u sad watch me break it down i run this town\ni breakdown the barz like a tic tac toe you a hoe my bro\njrizzy and brainbusta stay killin every rap nigga that ome\nface us girl put that face down and your ass up i be\nin da club getting my groove on i move like jagger\ni'm the one and only rapper that i be on my cool shit\nthis song y.u mad it's so legit that i let desperate bitches\nmassages my dick my topic is so sick y'all

In [4]:
# get all the chars in the sequence of text

chars = sorted(list(set(raw_text)))

In [5]:
# convert to dicts 

char_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_char = {i:ch for i,ch in enumerate(chars)}

In [6]:
# setup the variables used later 

n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: {} ".format(n_chars))
print ("Total Vocab: {}".format(n_vocab))

Total Characters: 6841 
Total Vocab: 43


In [7]:
# prepare the dataset of input to output pairs encoded as integers

seq_length = 100
dataX = []
dataY = []

for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])

n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  6741


In [8]:
dataY

[18,
 20,
 1,
 24,
 5,
 28,
 0,
 35,
 16,
 26,
 24,
 29,
 22,
 1,
 28,
 40,
 1,
 34,
 35,
 16,
 29,
 18,
 20,
 1,
 24,
 5,
 28,
 1,
 17,
 16,
 18,
 26,
 1,
 30,
 29,
 1,
 28,
 40,
 1,
 22,
 16,
 29,
 22,
 34,
 35,
 16,
 1,
 21,
 27,
 30,
 38,
 0,
 24,
 1,
 19,
 30,
 29,
 5,
 35,
 1,
 21,
 36,
 18,
 26,
 1,
 38,
 24,
 35,
 23,
 1,
 35,
 23,
 20,
 1,
 34,
 31,
 16,
 28,
 1,
 19,
 16,
 28,
 29,
 1,
 24,
 1,
 17,
 20,
 1,
 34,
 23,
 30,
 38,
 24,
 29,
 22,
 1,
 40,
 5,
 16,
 27,
 27,
 1,
 16,
 1,
 22,
 30,
 30,
 19,
 1,
 35,
 0,
 35,
 24,
 28,
 20,
 1,
 35,
 23,
 24,
 34,
 1,
 22,
 30,
 29,
 29,
 1,
 17,
 20,
 1,
 16,
 1,
 34,
 36,
 28,
 28,
 20,
 33,
 35,
 24,
 28,
 20,
 1,
 25,
 16,
 28,
 1,
 40,
 16,
 27,
 27,
 1,
 35,
 16,
 26,
 20,
 1,
 20,
 39,
 16,
 28,
 34,
 1,
 30,
 29,
 0,
 28,
 40,
 1,
 34,
 38,
 16,
 22,
 1,
 18,
 16,
 36,
 34,
 20,
 1,
 24,
 5,
 28,
 1,
 22,
 30,
 29,
 29,
 16,
 1,
 19,
 33,
 16,
 22,
 1,
 40,
 5,
 16,
 27,
 27,
 1,
 35,
 30,
 1,
 35,
 23,
 20,
 1,
 17,
 30,
 

In [9]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [10]:
# define the LSTM model

model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [11]:
# define the checkpoint

filepath="t4-eminem-weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [12]:
# load the model from previous epoch - 1ooth epoch
# load the network weights
# filename = "eminem-weights-improvement-13-2.2727.hdf5"
# model.load_weights(filename)
# model.compile(loss='categorical_crossentropy', optimizer='adam')

In [13]:
import time 
# your code here

t1 = time.time()

with tf.device('/gpu:0'):
    model.fit(X, y, epochs=100, batch_size=128,callbacks=callbacks_list)

t2 = time.time()
print("Training took:")
t2-t1

Epoch 1/100
6741/6741 [==============================] - 21s - loss: 3.1872    
Epoch 2/100
6741/6741 [==============================] - 18s - loss: 3.0710    
Epoch 3/100
6741/6741 [==============================] - 19s - loss: 3.0679    
Epoch 4/100
6741/6741 [==============================] - 19s - loss: 3.0651    
Epoch 5/100
6741/6741 [==============================] - 18s - loss: 3.0605    
Epoch 6/100
6741/6741 [==============================] - 19s - loss: 3.0279    
Epoch 7/100
6741/6741 [==============================] - 18s - loss: 2.9846    
Epoch 8/100
6741/6741 [==============================] - 19s - loss: 2.9430    
Epoch 9/100
6741/6741 [==============================] - 20s - loss: 2.8877    
Epoch 10/100
6741/6741 [==============================] - 18s - loss: 2.8624    
Epoch 11/100
6741/6741 [==============================] - 19s - loss: 2.8355    
Epoch 12/100
6741/6741 [==============================] - 18s - loss: 2.8183    
Epoch 13/100
6741/6741 [=============

6741/6741 [==============================] - 18s - loss: 1.7754    
Epoch 33/100
6741/6741 [==============================] - 19s - loss: 1.6789    
Epoch 34/100
6741/6741 [==============================] - 19s - loss: 1.6016    
Epoch 35/100
6741/6741 [==============================] - 19s - loss: 1.5126    
Epoch 36/100
6741/6741 [==============================] - 18s - loss: 1.4280    
Epoch 37/100
6741/6741 [==============================] - 19s - loss: 1.3415    
Epoch 38/100
6741/6741 [==============================] - 19s - loss: 1.2667    
Epoch 39/100
6741/6741 [==============================] - 19s - loss: 1.1824    
Epoch 40/100
6741/6741 [==============================] - 18s - loss: 1.1100    
Epoch 41/100
6741/6741 [==============================] - 19s - loss: 1.0231    
Epoch 42/100
6741/6741 [==============================] - 18s - loss: 0.9746    
Epoch 43/100
6741/6741 [==============================] - 19s - loss: 0.8942    
Epoch 44/100
6741/6741 [=================

6741/6741 [==============================] - 19s - loss: 0.1815    
Epoch 65/100
6741/6741 [==============================] - 19s - loss: 0.1738    
Epoch 66/100
6741/6741 [==============================] - 19s - loss: 0.1682    
Epoch 67/100
6741/6741 [==============================] - 19s - loss: 0.1577    
Epoch 68/100
6741/6741 [==============================] - 18s - loss: 0.1498    
Epoch 69/100
6741/6741 [==============================] - 18s - loss: 0.1319    
Epoch 70/100
6741/6741 [==============================] - 19s - loss: 0.1256    
Epoch 71/100
6741/6741 [==============================] - 19s - loss: 0.1253    
Epoch 72/100
6741/6741 [==============================] - 19s - loss: 0.1177    
Epoch 73/100
6741/6741 [==============================] - 19s - loss: 0.1111    
Epoch 74/100
6741/6741 [==============================] - 18s - loss: 0.1112    
Epoch 75/100
6741/6741 [==============================] - 19s - loss: 0.1074    
Epoch 76/100
6741/6741 [=================

6741/6741 [==============================] - 19s - loss: 0.0493    
Epoch 97/100
6741/6741 [==============================] - 19s - loss: 0.0524    
Epoch 98/100
6741/6741 [==============================] - 19s - loss: 0.0616    
Epoch 99/100
6741/6741 [==============================] - 18s - loss: 0.0669    
Epoch 100/100
6741/6741 [==============================] - 18s - loss: 0.0670    
Training took:


1912.0945937633514

In [36]:
import sys

# load text from the model

# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Rap Lyric Starting now:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

# generate characters
for i in range(3000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index) 
    pattern = pattern[1:len(pattern)]
print ("\n Hope it dropped some rhymes ;) ")

Rap Lyric Starting now:
" ting my sales on billboards yeah i shine like gold
y'all clusters i'm taking over the universe y'all "
 got them fake converses
it's so worthless for y'all cause to verse jordeezy and youngdeezy
cause they both go hard as shit jordeezy been dead and gone
youngdeezy taken over his soul yeah i'm an asshole but girls love it
im blasting shit
bitches on my johnson no magic shit
niggas talking crazy about my city i ain't having it
talking about new york fell off, ain't that some shit
if i ever see us falling off i bet im catching it
paper on your head through a phone call im faxing shit
oh and i get a quarter back and i sack a bitch
adding bitches subtracting bitches that mathamatic shitt
run up on your block and u git shocked when that static hit
im acting up, u r l smack it up
now u mad as fuck. you niggas softer than saggy nuts
im stacking up this dough hotter than dragon butt
my flow catching up you ugly couldn't bag a slut
i was never used to
being what i used